In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from tqdm import trange, tqdm, tqdm_notebook

In [15]:
academy = pd.read_csv('data/clean/academy.csv')
academy = academy.iloc[:, range(1,5)]
display(academy.head(5))

,이름,지번주소,위도,경도
0,클라비스음악학원,경기도 부천시 고강동 480-1번지 스카이빌딩 6층,37.525342,126.815545
1,라무지카아카데미음악학원,경기도 부천시 고강동 337번지 가람 5동 201호,37.523933,126.813580
2,고강비젼스터디보습학원,경기도 부천시 고강동 347-4번지 3층 일부,37.526758,126.816345
3,아이빈음악학원,경기도 부천시 고강동 389-4번지 3층 일부,37.526477,126.821402
4,고강보습학원,경기도 부천시 고강동 370-9번지 부강빌딩 3층 일부,37.527201,126.817407


In [16]:
candidate = pd.read_csv('output/cluster80.csv')
candidate = candidate.iloc[:, range(1,5)]
candidate.columns = ['군집', '인구밀도', '위도', '경도']
display(candidate)

,군집,인구밀도,위도,경도
0,0,677.363636,37.497041,126.767486
1,3,1207.307692,37.501440,126.750386
2,5,553.666667,37.496461,126.787226
3,6,599.444444,37.490873,126.790673
4,10,564.500000,37.480927,126.798087
...,...,...,...,...
76,7_1,475.250000,37.530072,126.808048
77,8_0,632.250000,37.469997,126.804220
78,8_1,677.857143,37.467883,126.800565
79,9_0,1836.500000,37.510268,126.756608


In [17]:
temp = gpd.read_file('data/bucheongrid/bucheonjsonjson.geojson')
temp = temp.dropna(axis=0)
축척 = sum([
    *(temp['geometry'].bounds['maxx'] - temp['geometry'].bounds['minx']),
    *(temp['geometry'].bounds['maxy'] - temp['geometry'].bounds['miny']),
    ]) / (2 * len(temp)) / 100
도보속도 = 133.1

In [18]:
minute = 5
거리 = minute * 133.1 / 100 * 60

In [7]:
def multipolygon_to_경도(x):
    경도, 위도 = x[0].exterior.xy 
    return np.array([x for x in 경도])
def multipolygon_to_위도(x):
    경도, 위도 = x[0].exterior.xy 
    return np.array([x for x in 위도])

In [8]:
def get_within_nminutes_infor(Data_input, Data_infor_input, name, value=False, geojson=False):
    """
    array을 이용하여 한번에 거리를 계산 후, 도보 3분이내 거리안에 해당하는 경우의
    수, 거리의 최소,평균,최대, 값의 최소,평균,최대를 계산합니다.
    
    geojson파일인 경우 멀티폴리곤에서 각각 경위도를 계산해주고 작업을 시행합니다.
    """
    
    Data = Data_input.copy()
    Data_infor = Data_infor_input.copy()
    
    print(name + "_작업중.............")
    
    ## geojson의 경우.. 지적도의 경우 경위도를 만들어 넣어 줄 거기 때문에 try로 함수입력
    if geojson:
        try:
            Data_infor['경도'] = Data_infor['geometry'].apply(multipolygon_to_경도) 
            Data_infor['위도'] = Data_infor['geometry'].apply(multipolygon_to_위도)        
        except:
            try:
                Data_infor['경도'] = Data_infor['geometry'].centroid.x
                Data_infor['위도'] = Data_infor['geometry'].centroid.y
            except:
                pass
            
    Len1 = len(Data)
    Len2 = len(Data_infor)
    
    tt_위도 = np.repeat(Data['위도'], Len2)
    tt_위도 = np.array(tt_위도).reshape(Len1, Len2)
    diff_위도 = tt_위도 - np.array(Data_infor['위도'])

    tt_경도 = np.repeat(Data['경도'], Len2)
    tt_경도 = np.array(tt_경도).reshape(Len1, Len2)
    diff_경도 = tt_경도 - np.array(Data_infor['경도'])
    
    diff_위도 = diff_위도**2
    diff_경도 = diff_경도**2
    dist = (diff_위도 + diff_경도)**(0.5)
    
    
    Count = []
    Dist_min = []
    Dist_mean = []
    Dist_max = []
    
    Val_min = []
    Val_mean = []
    Val_max = []
    
    
    
    for idx in tqdm(Data.index):
        if geojson:
            dist_idx = np.array([i.min() for i in dist[idx]])
        else:
            dist_idx = dist[idx]
        
        temp = Data_infor.loc[dist_idx < 거리 * 축척]           
        Count.append(len(temp))
        
        try:
            Dist_min.append(dist_idx[dist_idx < 거리 * 축척].min())
            Dist_mean.append(dist_idx[dist_idx < 거리 * 축척].mean())
            Dist_max.append(dist_idx[dist_idx < 거리 * 축척].max())
        except:
            ## 수정된 부분!
            Dist_min.append(0.0025)
            Dist_mean.append(0.0025)
            Dist_max.append(0.0025)
            
    Data[name + "_" + str(minute) + "분이내수"] = Count
    Data[name + "_" + str(minute) + "분이내거리최소"] = Dist_min
    Data[name + "_" + str(minute) + "분이내거리평균"] = Dist_mean
    Data[name + "_" + str(minute) + "분이내거리최대"] = Dist_max
        
    
    if value:
        for idx in tqdm(Data.index):
            if geojson:
                dist_idx = np.array([i.min() for i in dist[idx]])
            else:
                dist_idx = dist[idx]
                
            temp = Data_infor.loc[dist_idx < 거리 * 축척]
            Val_min.append(temp[value].min())
            Val_mean.append(temp[value].mean())
            Val_max.append(temp[value].max())
            
        Data[name + "_" + str(minute) + "분이내값최소"] = Val_min
        Data[name + "_" + str(minute) + "분이내값평균"] = Val_mean
        Data[name + "_" + str(minute) + "분이내값최대"] = Val_max
        Data = Data.fillna(0)
        
        return Data
    
    else:
        return Data

In [19]:
academy_result = get_within_nminutes_infor(Data_input = candidate, Data_infor_input = academy, name="학원")

학원_작업중.............


100%|████████████████████████████████████████████████████████████████████████████████| 81/81 [00:00<00:00, 3561.31it/s]


In [23]:
academy_result.head()

,군집,인구밀도,위도,경도,학원_5분이내수,학원_5분이내거리최소,학원_5분이내거리평균,학원_5분이내거리최대
0,0,677.363636,37.497041,126.767486,40,0.001019,0.002794,0.004017
1,3,1207.307692,37.501440,126.750386,126,0.001493,0.002893,0.004002
2,5,553.666667,37.496461,126.787226,20,0.001662,0.003284,0.004080
3,6,599.444444,37.490873,126.790673,36,0.000812,0.002476,0.003918
4,10,564.500000,37.480927,126.798087,4,0.002960,0.003258,0.003743


In [29]:
academy_result = academy_result[['군집', '위도', '경도', '학원_5분이내거리최소']]

In [32]:
academy_result.to_csv('output/var_cluster_academy.csv')